**1. Import de Librerías necesarias**

In [37]:
import os
import subprocess
import pandas as pd
from datetime import date

from pyspark import SparkContext, HiveContext
from pyspark.sql import DataFrame, SQLContext, SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import to_timestamp
from pyspark.sql import functions as f
from pyspark.sql.functions import percentile_approx
from pyspark.sql.functions import col, substring

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

In [2]:
ss = SparkSession.builder.appName("bigdatita").getOrCreate()

24/12/12 02:53:38 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


**2. Lectura de los archivos**

In [3]:
!hdfs dfs -ls /user/bigdatita/chicago

Found 2 items
-rw-r--r--   2 navitq hadoop 2689650394 2024-12-12 01:22 /user/bigdatita/chicago/Taxi_Trips_-_2023_20241130.csv
-rw-r--r--   2 navitq hadoop 2253224851 2024-12-12 01:22 /user/bigdatita/chicago/Taxi_Trips__2024-__20241210.csv


In [4]:
ruta = '/user/bigdatita/chicago'
cmd = f'hdfs dfs -ls {ruta}'
lst = str(subprocess.check_output(cmd, shell=True)).strip().split('\\n')
lst = [x.split(' ')[-1] for x in lst]
lst = ['hdfs://%s'%x for x in lst if x!="items" if x!="'"]
lst

['hdfs:///user/bigdatita/chicago/Taxi_Trips_-_2023_20241130.csv',
 'hdfs:///user/bigdatita/chicago/Taxi_Trips__2024-__20241210.csv']

**3. Lectura de archivos desde HDFS a Spark**

In [5]:
%%time
for i, file in enumerate(lst):
    print(file)
    if i==0:
        df = ss.read.csv(file, header=True, inferSchema=False)
    else:
        df = df.union(ss.read.csv(file, header=True, inferSchema=False))

hdfs:///user/bigdatita/chicago/Taxi_Trips_-_2023_20241130.csv


hdfs:///user/bigdatita/chicago/Taxi_Trips__2024-__20241210.csv


CPU times: user 8.76 ms, sys: 11.2 ms, total: 19.9 ms
Wall time: 11.5 s


In [6]:
df.printSchema()

root
 |-- Trip ID: string (nullable = true)
 |-- Taxi ID: string (nullable = true)
 |-- Trip Start Timestamp: string (nullable = true)
 |-- Trip End Timestamp: string (nullable = true)
 |-- Trip Seconds: string (nullable = true)
 |-- Trip Miles: string (nullable = true)
 |-- Pickup Census Tract: string (nullable = true)
 |-- Dropoff Census Tract: string (nullable = true)
 |-- Pickup Community Area: string (nullable = true)
 |-- Dropoff Community Area: string (nullable = true)
 |-- Fare: string (nullable = true)
 |-- Tips: string (nullable = true)
 |-- Tolls: string (nullable = true)
 |-- Extras: string (nullable = true)
 |-- Trip Total: string (nullable = true)
 |-- Payment Type: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Pickup Centroid Latitude: string (nullable = true)
 |-- Pickup Centroid Longitude: string (nullable = true)
 |-- Pickup Centroid Location: string (nullable = true)
 |-- Dropoff Centroid Latitude: string (nullable = true)
 |-- Dropoff Centroid

In [7]:
date_format = 'MM/dd/yyyy hh:mm:ss a'

df = df.withColumn('Trip Start Timestamp', to_timestamp(df['Trip Start Timestamp'], date_format).cast(TimestampType()))
df = df.withColumn('Trip End Timestamp', to_timestamp(df['Trip End Timestamp'], date_format).cast(TimestampType()))

## Filtrando información

In [8]:
df = df[(df['Trip Start Timestamp'] <= '2024-08-31')]

In [9]:
df.select(f.max("Trip Start Timestamp")).show()

+-------------------------+
|max(Trip Start Timestamp)|
+-------------------------+
|      2024-08-31 00:00:00|
+-------------------------+



In [10]:
for var in ['Fare', 'Tips', 'Tolls', 'Extras', 'Trip Total']:
    df = df.withColumn(var, df[var].cast('float'))

In [11]:
df.printSchema()

root
 |-- Trip ID: string (nullable = true)
 |-- Taxi ID: string (nullable = true)
 |-- Trip Start Timestamp: timestamp (nullable = true)
 |-- Trip End Timestamp: timestamp (nullable = true)
 |-- Trip Seconds: string (nullable = true)
 |-- Trip Miles: string (nullable = true)
 |-- Pickup Census Tract: string (nullable = true)
 |-- Dropoff Census Tract: string (nullable = true)
 |-- Pickup Community Area: string (nullable = true)
 |-- Dropoff Community Area: string (nullable = true)
 |-- Fare: float (nullable = true)
 |-- Tips: float (nullable = true)
 |-- Tolls: float (nullable = true)
 |-- Extras: float (nullable = true)
 |-- Trip Total: float (nullable = true)
 |-- Payment Type: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Pickup Centroid Latitude: string (nullable = true)
 |-- Pickup Centroid Longitude: string (nullable = true)
 |-- Pickup Centroid Location: string (nullable = true)
 |-- Dropoff Centroid Latitude: string (nullable = true)
 |-- Dropoff Centroi

**4. Operaciones básicas**

In [12]:
from IPython.display import display

In [13]:
describe_df = df.describe().toPandas()

24/12/12 02:54:32 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


## Análisis exploratorio

In [14]:
subset = [
    
    'summary',
    'Trip Seconds', 'Trip Miles',
    'Pickup Census Tract', 'Dropoff Census Tract', 'Pickup Community Area',
    'Dropoff Community Area', 'Fare', 'Tips', 'Tolls', 'Extras',
    'Trip Total', 'Payment Type', 'Company', 'Pickup Centroid Latitude',
    'Pickup Centroid Longitude', 'Pickup Centroid Location',
    'Dropoff Centroid Latitude', 'Dropoff Centroid Longitude',
    'Dropoff Centroid  Location'
]

display(describe_df[subset].round(2))

,summary,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,Tips,Tolls,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
0,count,10753418,10755450,4715559,4579195,10395504,9728878,10731465,10731465,10731465,10731465,10731465,10755497,10755497,10400001,10400001,10400001,9793910,9793910,9793910
1,mean,1253.8432065042018,6.611410067454635,1.7031507928872877E10,1.703141496867211E10,35.790761563845294,26.238793517608094,22.538056911430935,2.9734957981665007,0.0495877094770661,2.274319610613869,28.014873584051113,None,None,41.902169394980184,-87.70184576171364,None,41.893577867825144,-87.66223343453349,None
2,stddev,1675.5977047159922,7.849773679995619,375165.19895197486,344238.57901968947,26.3030986498151,20.95742682723646,26.238946021802896,4.293365694862301,10.821039066251112,18.14324821362521,38.6419479970113,None,None,0.06372773833722475,0.11410234491471953,None,0.05732473692013584,0.07348264270532737,None
3,min,0,0,17031010100,17031010100,1,1,0.0,0.0,0.0,0.0,0.0,Cash,2733 - 74600 Benny Jona,41.650221676,-87.531386257,POINT (-87.5313862567 41.7204632831),41.650221676,-87.534902901,POINT (-87.5349029012 41.707311449)
4,max,9999,99.99,17031980100,17031980100,9,9,9999.75,500.0,6666.66,9693.78,9999.75,Unknown,Wolley Taxi,42.021223593,-87.913624596,POINT (-87.913624596 41.9802643146),42.021223593,-87.913624596,POINT (-87.913624596 41.9802643146)


In [15]:
%%time
deciles = df.selectExpr("percentile_approx(Fare, array(0.0, 0.01, 0.02, 0.03, 0.97, 0.98, 0.99, 1.0)) as Percentiles")
deciles.show(truncate=False)

+-----------------------------------------------------+
|Percentiles                                          |
+-----------------------------------------------------+
|[0.0, 3.25, 3.25, 3.25, 58.25, 64.25, 73.25, 9999.75]|
+-----------------------------------------------------+

CPU times: user 26.7 ms, sys: 15.9 ms, total: 42.6 ms
Wall time: 25.4 s


## Valores nulo

In [23]:
from pyspark.sql.functions import col, isnan, when, count

# Columnas numéricas
numeric_columns = [c for c, t in df.dtypes if t in ["double", "float"]]

# Columnas no numéricas
other_columns = [c for c, t in df.dtypes if t not in ["double", "float"]]

# Contar valores faltantes para columnas numéricas
numeric_missing = [
    count(when(isnan(col(c)) | col(c).isNull(), c)).alias(c) for c in numeric_columns
]

# Contar valores faltantes para columnas no numéricas
other_missing = [
    count(when(col(c).isNull(), c)).alias(c) for c in other_columns
]

# Combinar resultados
missings_df = df.select(numeric_missing + other_missing).toPandas()

In [24]:
display(missings_df.round())

,Fare,Tips,Tolls,Extras,Trip Total,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
0,24032,24032,24032,24032,24032,0,0,0,121,2079,47,6039938,6176302,359993,1026619,0,0,355496,355496,355496,961587,961587,961587


In [25]:
total = df.count()
missings_df = missings_df.T / total * 100


In [26]:
missings_df=missings_df.reset_index().sort_values(0, ascending=False)
missings_df

,index,0
12,Dropoff Census Tract,57.424608
11,Pickup Census Tract,56.156754
14,Dropoff Community Area,9.545063
21,Dropoff Centroid Longitude,8.940424
20,Dropoff Centroid Latitude,8.940424
22,Dropoff Centroid Location,8.940424
13,Pickup Community Area,3.347061
19,Pickup Centroid Location,3.305249
18,Pickup Centroid Longitude,3.305249
17,Pickup Centroid Latitude,3.305249


## Missings en las variables de Census Tract

In [27]:
missings_df

,index,0
12,Dropoff Census Tract,57.424608
11,Pickup Census Tract,56.156754
14,Dropoff Community Area,9.545063
21,Dropoff Centroid Longitude,8.940424
20,Dropoff Centroid Latitude,8.940424
22,Dropoff Centroid Location,8.940424
13,Pickup Community Area,3.347061
19,Pickup Centroid Location,3.305249
18,Pickup Centroid Longitude,3.305249
17,Pickup Centroid Latitude,3.305249


In [33]:
# Total de Census con missings
print(f'Missings Pickup: {missings_df[missings_df["index"] == "Dropoff Census Tract"].values}')
print(f'Missings Dropoff: {missings_df[missings_df["index"] == "Pickup Census Tract"].values}')

Missings Pickup: [['Dropoff Census Tract' 57.424608086451045]]
Missings Dropoff: [['Pickup Census Tract' 56.15675407654337]]


**Queremos ver cuáles son las diferencias de sectores con los que contamos para obtener si hay pickup o dropoff fuera de Chicago.**

In [34]:
# Número de valores únicos en la columna "Pickup Census Tract" en PySpark
pickup_unique_count = df.select("Pickup Census Tract").distinct().count()
print(f'Número de valores únicos en "Pickup Census Tract": {pickup_unique_count}')


Número de valores únicos en "Pickup Census Tract": 778


In [35]:
# Número de valores únicos en la columna "Pickup Census Tract" en PySpark
pickup_unique_count = df.select("Dropoff Census Tract").distinct().count()
print(f'Número de valores únicos en "Dropoff Census Tract": {pickup_unique_count}')


Número de valores únicos en "Dropoff Census Tract": 923


In [43]:
# Obtener los valores únicos truncados de "Dropoff Census Tract"
Pickup_unique = df.select(substring(col("Pickup Census Tract"), 1, 5).alias("truncated_Pickup")) \
    .distinct()

# Imprimir el número de valores únicos
print(f'Número de valores únicos en los primeros 5 caracteres de "Pickup Census Tract": {Pickup_unique.count()}')

# Imprimir los valores únicos
unique_values_Pickup = Pickup_unique.collect()
print("Valores únicos en los primeros 5 caracteres de 'Pickup Census Tract':")
for row in unique_values_Pickup:
    print(row["truncated_Pickup"])

Número de valores únicos en los primeros 5 caracteres de "Pickup Census Tract": 2


Valores únicos en los primeros 5 caracteres de 'Pickup Census Tract':
17031
None


In [41]:
# Obtener los valores únicos truncados de "Dropoff Census Tract"
dropoff_unique = df.select(substring(col("Dropoff Census Tract"), 1, 5).alias("truncated_dropoff")) \
    .distinct()

# Imprimir el número de valores únicos
print(f'Número de valores únicos en los primeros 5 caracteres de "Dropoff Census Tract": {dropoff_unique.count()}')

# Imprimir los valores únicos
unique_values = dropoff_unique.collect()
print("Valores únicos en los primeros 5 caracteres de 'Dropoff Census Tract':")
for row in unique_values:
    print(row["truncated_dropoff"])

Número de valores únicos en los primeros 5 caracteres de "Dropoff Census Tract": 2


Valores únicos en los primeros 5 caracteres de 'Dropoff Census Tract':
17031
None


**Observaciones**

El registro de sectores en Chicago es únicamente de 5 dígitos y el registro de los Census Tract estan dados en 11 dígitos, si reducimos estos digitos y unicamente nos quedamos con los 5 útiles podremos darnos cuentas que todos nuestros viajes son dentro del Condado de Cook.

**Conclusión**

Por la cantidad de datos faltantes en Census Tract tanto en Pickup como Dropoff, y además al ser una variable unaria ya que la información contenida está en función del mismo sector, la variable resulta poco relevante para el análisis. Adicionalmente, carece de sentido aplicar imputación. Por lo que se deben eliminar dichas variables.

In [44]:
columns_to_exclude = ['Pickup Census Tract', 'Dropoff Census Tract']

In [45]:
# Eliminar columnas en PySpark
df = df.drop(*columns_to_exclude)
# Verificar el esquema para confirmar que se eliminaron las columnas
df.printSchema()

root
 |-- Trip ID: string (nullable = true)
 |-- Taxi ID: string (nullable = true)
 |-- Trip Start Timestamp: timestamp (nullable = true)
 |-- Trip End Timestamp: timestamp (nullable = true)
 |-- Trip Seconds: string (nullable = true)
 |-- Trip Miles: string (nullable = true)
 |-- Pickup Community Area: string (nullable = true)
 |-- Dropoff Community Area: string (nullable = true)
 |-- Fare: float (nullable = true)
 |-- Tips: float (nullable = true)
 |-- Tolls: float (nullable = true)
 |-- Extras: float (nullable = true)
 |-- Trip Total: float (nullable = true)
 |-- Payment Type: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Pickup Centroid Latitude: string (nullable = true)
 |-- Pickup Centroid Longitude: string (nullable = true)
 |-- Pickup Centroid Location: string (nullable = true)
 |-- Dropoff Centroid Latitude: string (nullable = true)
 |-- Dropoff Centroid Longitude: string (nullable = true)
 |-- Dropoff Centroid  Location: string (nullable = true)



#### Missings en Trip Total

**Recalculamos el total de valores ausentes con los que cuenta ahora el conjunto de datos.**

In [46]:
from pyspark.sql.functions import col, isnan, when, count

# Columnas numéricas
numeric_columns = [c for c, t in df.dtypes if t in ["double", "float"]]

# Columnas no numéricas
other_columns = [c for c, t in df.dtypes if t not in ["double", "float"]]

# Contar valores faltantes para columnas numéricas
numeric_missing = [
    count(when(isnan(col(c)) | col(c).isNull(), c)).alias(c) for c in numeric_columns
]

# Contar valores faltantes para columnas no numéricas
other_missing = [
    count(when(col(c).isNull(), c)).alias(c) for c in other_columns
]

# Combinar resultados
missings_df = df.select(numeric_missing + other_missing).toPandas()

In [47]:
display(missings_df.round())

,Fare,Tips,Tolls,Extras,Trip Total,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Community Area,Dropoff Community Area,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
0,24032,24032,24032,24032,24032,0,0,0,121,2079,47,359993,1026619,0,0,355496,355496,355496,961587,961587,961587


In [48]:
total = df.count()
missings_df = missings_df.T / total * 100

In [49]:
missings_df=missings_df.reset_index().sort_values(0, ascending=False)
missings_df

,index,0
12,Dropoff Community Area,9.545063
20,Dropoff Centroid Location,8.940424
19,Dropoff Centroid Longitude,8.940424
18,Dropoff Centroid Latitude,8.940424
11,Pickup Community Area,3.347061
17,Pickup Centroid Location,3.305249
16,Pickup Centroid Longitude,3.305249
15,Pickup Centroid Latitude,3.305249
1,Tips,0.223439
0,Fare,0.223439


In [56]:
# Filtrar filas donde 'Trip Total' es nulo
filtered_df = df.filter(col('Trip Total').isNull())

# Calcular estadísticas descriptivas
summary_df=filtered_df.describe()

In [57]:
summary_pandas = summary_df.toPandas()

  summary                                   Trip ID                                            Taxi ID        Trip Seconds         Trip Miles Pickup Community Area Dropoff Community Area  Fare  Tips Tolls Extras Trip Total Payment Type      Company Pickup Centroid Latitude Pickup Centroid Longitude             Pickup Centroid Location Dropoff Centroid Latitude Dropoff Centroid Longitude           Dropoff Centroid  Location
0   count                                     24032                                              24032               23325              24031                 23299                  21655     0     0     0      0          0        24032        24032                    23341                     23341                                23341                     21902                      21902                                21902
1    mean                                      None                                               None  1233.2925616291532  6.917065457117887     

In [58]:
# Mostrar como tabla en Pandas
summary_pandas

,summary,Trip ID,Taxi ID,Trip Seconds,Trip Miles,Pickup Community Area,Dropoff Community Area,Fare,Tips,Tolls,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
0,count,24032,24032,23325,24031,23299,21655,0,0,0,0,0,24032,24032,23341,23341,23341,21902,21902,21902
1,mean,None,None,1233.2925616291532,6.917065457117887,35.27936821322803,31.398522281228352,None,None,None,None,None,None,None,41.90692753224614,-87.70547707558165,None,41.90038405208939,-87.68980451077122,None
2,stddev,None,None,1461.3323671358514,8.531191479931204,26.41426167221771,24.360003103664383,None,None,None,None,None,None,None,0.05820874068062427,0.11432610482539693,None,0.05836584836986091,0.10119648310467881,None
3,min,00047fc448f982b3394475afe08fe7b24c422f42,0044e6c0d091476299b99345501f756b23632a96cbaf40...,0,0,1,1,None,None,None,None,None,Cash,5 Star Taxi,41.660136051,-87.534902901,POINT (-87.5349029012 41.707311449),41.660136051,-87.534902901,POINT (-87.5349029012 41.707311449)
4,max,fffc1f8edbe541ea864534c94abcd137b7f219a9,ffc2a618ad19a63fa5e9ce8b1bd68656172b2eac505f0a...,999,95.95,9,9,None,None,None,None,None,Unknown,Top Cab,42.016046492,-87.913624596,POINT (-87.913624596 41.9802643146),42.016046492,-87.913624596,POINT (-87.913624596 41.9802643146)


**Como el objetivo principal es generar una TAD por taxi (no por viaje), es importante identificar si existen taxis que tengan algún problema al registrar su total de pagos.**

In [64]:
taxistas_con_valores_faltantes= df.filter(col('Trip Total').isNull()).groupby('Taxi ID').agg(f.count("*").alias('Cantidad de registros con Trip Total faltante'))
taxistas_con_valores_faltantes=taxistas_con_valores_faltantes.orderBy(col('Cantidad de registros con Trip Total faltante').desc())
taxistas_con_valores_faltantes.show()

+--------------------+---------------------------------------------+
|             Taxi ID|Cantidad de registros con Trip Total faltante|
+--------------------+---------------------------------------------+
|5d7636d4b116cb207...|                                         1263|
|ed7090d32800eec66...|                                         1124|
|535cbdf78f906572b...|                                         1079|
|87a21e5ac34dbe38d...|                                         1032|
|2407ff99e367ceb46...|                                          974|
|ea1095a215ac661a9...|                                          661|
|3671b00a8d3d78e49...|                                          628|
|5e48748b582fe3c16...|                                          555|
|18c20740c9cbe78a0...|                                          532|
|180309cc29892cd16...|                                          469|
|4b2d5e8743e0e17dd...|                                          452|
|286e28223cab05d01...|            

In [65]:
# Calcular el total de registros por taxista
total_registros_por_taxista=df.groupby('Taxi ID').agg(f.count("*").alias('Total de registros por taxista'))
# Cruzar el DataFrame de taxistas_con_valores_faltantes con el DataFrame total_registros_por_taxista
taxistas_con_valores_faltantes=taxistas_con_valores_faltantes.join(total_registros_por_taxista, on='Taxi ID',how='inner')
# Calcular el porcentaje de registros faltantes por taxista
taxistas_con_valores_faltantes=taxistas_con_valores_faltantes.withColumn('Porcentaje de registros faltantes',(col('Cantidad de registros con Trip Total faltante')/col('Total de registros por taxista'))*100)
taxistas_con_valores_faltantes=taxistas_con_valores_faltantes.orderBy(col('Porcentaje de registros faltantes').desc())
taxistas_con_valores_faltantes.show()

+--------------------+---------------------------------------------+------------------------------+---------------------------------+
|             Taxi ID|Cantidad de registros con Trip Total faltante|Total de registros por taxista|Porcentaje de registros faltantes|
+--------------------+---------------------------------------------+------------------------------+---------------------------------+
|2407ff99e367ceb46...|                                          974|                          2432|                40.04934210526316|
|8cc4bb2cb443a9e7a...|                                            2|                             6|                33.33333333333333|
|535cbdf78f906572b...|                                         1079|                          5389|                20.02226758211171|
|ed7090d32800eec66...|                                         1124|                          5898|               19.057307561885384|
|4b2d5e8743e0e17dd...|                                        

**Podemos darnos cuenta que el taxista con la mayor afectación es de más del 40% de sus registros, por lo que se recomienda pornerse en contacto con los conductores priorizando aquellos que tengan mayor afectación (mayor cantidad de montos sin registro).**

In [66]:
df=df.filter(col('Trip Total').isNotNull())

In [67]:
Trip_Total_null=df.filter(col('Trip Total').isNull()).count()
Trip_Total_null

0

Al eliminar los registros que no cuentan con 'Trip Total' también se eliminan todos aquellos registros relacionados el desglose de montos, es decir, los montos asociados a las variables 'Fare', 'Extras', 'Tolls' y 'Tips'.

#### Missings en Trip Seconds

In [68]:
Trip_Seconds=df.filter(col('Trip Seconds').isNull()).count()
Trip_Seconds

1372

In [69]:
Trip_Seconds_porcentaje=(Trip_Trip_Seconds*100)/df.count()
Trip_Seconds_porcentaje

0.012784834130288828

Debido a que contamos con valores ausentes en la variable 'Trip Seconds' equivalentes al 0.0143% de los datos y a que no contamos con suficiente informacion para imputar valores basándonos en calcular la diferencia en segundos de los viajes entre las variables 'Trip End Timestamp' y 'Trip Start Timestamp' para interpolar los datos faltantes ya que estos no corresponden por completo a la variable 'Trip Seconds', y que adicionalmente, en algunos casos la diferencia es considerable. Se concluye que lo más recomendable es eliminar estos registros.

In [70]:
df=df.filter(col('Trip Seconds').isNotNull())

In [71]:
Trip_Seconds=df.filter(col('Trip Seconds').isNull()).count()
Trip_Seconds

0

#### Missings en Trip Miles

In [73]:
Trip_Miles=df.filter(col('Trip Miles').isNull())
Trip_Miles.count()

44

Revisamos cómo se comportan estos valores para los registros en Trip Miles.

In [74]:
Trip_Miles_describe=Trip_Miles.describe().toPandas()
Trip_Miles_describe

,summary,Trip ID,Taxi ID,Trip Seconds,Trip Miles,Pickup Community Area,Dropoff Community Area,Fare,Tips,Tolls,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
0,count,44,44,44,0,37,39,44,44,44,44,44,44,44,37,37,37,39,39,39
1,mean,None,None,2545.6136363636365,None,23.89189189189189,24.307692307692307,357.57863688468933,3.6218182173642246,0.0,0.2840909090909091,361.65500099008733,None,None,41.924520748297304,-87.70792070381081,None,41.930831966102566,-87.71492091166668,None
2,stddev,None,None,6752.581732682243,None,23.342597903337083,23.53625852991602,1498.7815182185643,13.584712604241323,0.0,0.7578866522045707,1498.5627840885736,None,None,0.0454379922928036,0.09632599573449512,None,0.05147213511446959,0.0875831463209836,None
3,min,056cf90618eef5b60f19f5720ae8115abac33834,0f831bff43d83f396f2e4950126c6137dcdb60fb4c8580...,0,None,10,10,0.0,0.0,0.0,0.0,0.0,Cash,5 Star Taxi,41.79259236,-87.612945414,POINT (-87.6129454143 41.8919715078),41.79259236,-87.612945414,POINT (-87.6129454143 41.8919715078)
4,max,f7cbb55a621eaa8d9279b5822cd5cb457899a1f3,f621299fa67bd8263e781be39bd954acda79ca9c958849...,9684,None,8,8,9999.75,65.01,0.0,4.0,9999.75,Prcard,"Taxicab Insurance Agency, LLC",41.993930128,-87.913624596,POINT (-87.913624596 41.9802643146),42.001571027,-87.913624596,POINT (-87.913624596 41.9802643146)


Taxistas con afectación

In [76]:
taxistas_con_valores_faltantes= df.filter(col('Trip Miles').isNull()).groupby('Taxi ID').agg(f.count("*").alias('Cantidad de registros con Trip Miles faltante'))
taxistas_con_valores_faltantes=taxistas_con_valores_faltantes.orderBy(col('Cantidad de registros con Trip Miles faltante').desc())
taxistas_con_valores_faltantes.count()

29

In [77]:
total_taxis=df.groupby('Taxi ID').agg(f.count("*").alias('Cantidad total de taxis'))
total_taxis.count()

3790

In [78]:
(taxistas_con_valores_faltantes.count()/total_taxis.count())*100

0.7651715039577837

**Observación**

Podemos ver que afectamos solamente a 29 diferentes Taxis de un total de 3,709 Taxis que representa un 0.7651715039577837% del total de los Taxis por lo que es posible eliminar estos registros.

In [79]:
df=df.filter(col('Trip Miles').isNotNull())

In [80]:
Trip_Miles=df.filter(col('Trip Miles').isNull()).count()
Trip_Miles

0

#### Missings en Coordenadas

Finalmente, los *missings* restantes deben corresponder únicamente a los que están en función de las coordenadas del viaje.

In [96]:
from pyspark.sql.functions import col, isnan, when, count

# Columnas numéricas
numeric_columns = [c for c, t in df.dtypes if t in ["double", "float"]]

# Columnas no numéricas
other_columns = [c for c, t in df.dtypes if t not in ["double", "float"]]

# Contar valores faltantes para columnas numéricas
numeric_missing = [
    count(when(isnan(col(c)) | col(c).isNull(), c)).alias(c) for c in numeric_columns
]

# Contar valores faltantes para columnas no numéricas
other_missing = [
    count(when(col(c).isNull(), c)).alias(c) for c in other_columns
]

# Combinar resultados
missings_df = df.select(numeric_missing + other_missing).toPandas()

In [97]:
missings_df=missings_df.T

In [98]:
missings_df=missings_df.reset_index()
missings_df=missings_df.rename(columns={0:'missings'})
missings_df

,index,missings
0,Fare,0
1,Tips,0
2,Tolls,0
3,Extras,0
4,Trip Total,0
5,Trip ID,0
6,Taxi ID,0
7,Trip Start Timestamp,0
8,Trip End Timestamp,0
9,Trip Seconds,0


In [99]:
missings_df=missings_df[missings_df['missings']>0]
missings_df

,index,missings
11,Pickup Community Area,359070
12,Dropoff Community Area,1023851
15,Pickup Centroid Latitude,354616
16,Pickup Centroid Longitude,354616
17,Pickup Centroid Location,354616
18,Dropoff Centroid Latitude,959072
19,Dropoff Centroid Longitude,959072
20,Dropoff Centroid Location,959072


In [101]:
total = df.count()
missings_df['porcentaje_nulos'] = missings_df['missings'] / total * 100
missings_df.sort_values(0, ascending=False)

/tmp/ipykernel_11826/2598878292.py:2: SettingWithCopyWarning:                   
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missings_df['porcentaje_nulos'] = missings_df['missings'] / total * 100


KeyError: 0

In [ ]:
Tomamos 'Dropoff Community Area' como filtro a explorar al ser la variable con la mayor cantidad de registros faltantes al momento.

In [ ]:
taxi_df.dropna(subset=['Trip Total'], inplace=True)

In [ ]:
taxi_df['Trip Total'].isna().sum()

In [ ]:
%%time
df2 = df.filter((df['Fare']>=3.25) & (df['Fare']<61) & (df['Trip Start Timestamp']>date(2022,1,1)))

In [ ]:
%%time
df2.count()

In [ ]:
%%time
df2.describe('Fare').show()

In [ ]:
df2.select('Taxi ID', 'Trip Start Timestamp', 'Trip End Timestamp', 'Fare').show(10)

**5. Funciones definidas por el usuario**

In [ ]:
f = udf(lambda x:x.strftime('%Y%m'))

In [ ]:
%%time
df.withColumn('Month', f(df['Trip Start Timestamp'])).select('Month', 'Trip Start Timestamp', 'Fare').show(5)

In [ ]:
df3 = df2.withColumn('Month', f(df2['Trip Start Timestamp']))

In [ ]:
from pyspark.sql import functions as F

In [ ]:
%%time
df3.groupby('Month').agg(F.min('Fare'), F.mean('Fare'), F.max('Fare')).orderBy('Month').show()

**6. Uso de SQL**

In [ ]:
sqlContext = SQLContext(sc)

In [ ]:
df2.registerTempTable("taxi_trips")

In [ ]:
query = """
SELECT
    year('Trip Start Timestamp')*100 + month('Trip Start Timestamp') as fh,
    count(*) as casos
FROM
    taxi_trips
GROUP BY
    year('Trip Start Timestamp')*100 + month('Trip Start Timestamp')
ORDER BY
    fh
"""

In [ ]:
query = """SELECT * FROM taxi_trips LIMIT 10;"""

In [ ]:
%%time
sqlContext.sql(query).show()